In [1]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from goose3 import Goose
import time
import requests
import random
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
program_languages=['bash','r','python','java','c++','c#', 'f#', 'ruby','perl','matlab','javascript','scala','php']
analysis_software=['alteryx', 'excel','tableau', 'power bi', 'powerbi', 'qlikview', 'qlik','d3.js','sas','sap','spss','d3','saas','pandas','numpy','scipy','sps','spotfire','scikits.learn','splunk','powerpoint','h2o']
bigdata_tool=['hadoop','mapreduce','spark','pig','hive','shark','oozie','zookeeper','flume','mahout']
databases=['sql','nosql','hbase','cassandra','mongodb','mysql','mssql','postgresql','oracle db','rdbms']
machine_learning = ['keras', 'scikit-learn', 'tensorflow', 'caffe', 'datarobot', 'theano', 'torch']
overall_dict = program_languages + analysis_software + bigdata_tool + databases + machine_learning

In [3]:
def keywords_extract(url):
    g = Goose()
    article = g.extract(url=url)
    text = article.cleaned_text
    text = re.sub("[^a-zA-Z+3]"," ", text) #get rid of things that aren't words; 3 for d3 and + for c++
    text = text.lower().split()
    stops = set(stopwords.words("english")) #filter out stop words in english language
    text = [w for w in text if not w in stops]
    text = list(set(text))
    keywords = [str(word) for word in text if word in overall_dict]
    return keywords

In [4]:
def keywords_f(soup_obj):
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    text = soup_obj.get_text() 
    lines = (line.strip() for line in text.splitlines()) # break into line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    try:
        text = text.decode('utf-8') # Need this as some websites aren't formatted
    except:                                                          
        return                                                       
    text = re.sub("[^a-zA-Z+3]"," ", text)  
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    text = text.lower()
    text = re.sub(('power bi'), 'powerbi', text) # catch power bi before tokenization
    text = text.split() 
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    text = list(set(text)) #only care about if a word appears, don't care about the frequency
    keywords = [str(word) for word in text if word in overall_dict] #if a skill keyword is found, return it.
    return keywords

In [ ]:
def communication_skills_f(soup_obj):
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    text = soup_obj.get_text() 
    lines = (line.strip() for line in text.splitlines()) # break into line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    try:
        text = text.decode('utf-8') # Need this as some websites aren't formatted
    except:                                                          
        return                                                       
    text = re.sub("[^a-zA-Z+3]"," ", text)  
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    text = text.lower()
    skills = [word for word in soft_skills if word in text] 
    return skills

In [5]:
base_url = "http://www.indeed.nl"    
#change the start_url can scrape different cities.
start_url = "http://www.indeed.nl/jobs?q=data+scientist&l="
resp = requests.get(start_url)
start_soup = BeautifulSoup(resp.content, 'lxml', from_encoding='utf-8')
urls=start_soup.find_all('a',{'rel':'nofollow','target':'_blank'})
urls = [link['href'] for link in urls] 
num_found = start_soup.find(id = 'searchCount').string.encode('utf-8').split() #this returns the total number of results
num_jobs = num_found[-1].decode('utf-8').split(',')
num_jobs = [x.replace('.', '') for x in num_jobs]

if len(num_jobs)>=2:
    num_jobs = int(num_jobs[0]) * 1000 + int(num_jobs[1])
else:
    num_jobs = int(num_jobs[0])
num_pages = int(num_jobs/10) #calculates how many pages needed to do the scraping

job_keywords=[]
print('There are %d jobs found and we need to extract %d pages.'%(num_jobs,num_pages))
print('extracting first page of job searching results')

There are 1073 jobs found and we need to extract 107 pages.
extracting first page of job searching results


In [6]:
# # prevent the driver stopping due to the unexpectedAlertBehaviour.
# webdriver.DesiredCapabilities.FIREFOX["unexpectedAlertBehaviour"] = "accept"
# get_info = True
# driver = webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
# # set a page load time limit so that don't have to wait forever if the links are broken.
# driver.set_page_load_timeout(15)
# for i in range(len(urls)):
#     get_info = True
#     try:
#         driver.get(base_url+urls[i])
#     except TimeoutException:
#         get_info = False
#         continue
#     j = random.randint(1000,2200)/1000.0
#     time.sleep(j) #waits for a random time so that the website don't consider you as a bot
#     if get_info:
#         soup=BeautifulSoup(driver.page_source, 'lxml')
#         print('extracting %d job keywords...' % i)
#         soup_string = str(soup)
#         single_job = keywords_f(soup)
#         print(single_job,len(soup))
#         print(driver.current_url)
#         job_keywords.append([driver.current_url,single_job])

extracting 0 job keywords...
['java'] 1
https://www.indeed.nl/vacature-bekijken?jk=43cf6a283215a0f2&tk=1ckph758o14l40lu&from=serp&alid=3&advn=2421485629829485
extracting 1 job keywords...
['hadoop', 'r', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckph77ke14l428h&from=serp&alid=3&advn=2327986768228300
extracting 2 job keywords...
['spss', 'r', 'sql', 'python', 'excel'] 1
https://www.indeed.nl/vacature-bekijken?jk=2a3d7f6df1fca6ee&tk=1ckph7aar14l41nl&from=serp&alid=3&advn=9110344694364511
extracting 3 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=90f9baa202e07bbe&q=data+scientist&from=web&advn=4203524812083670&sjdu=e0gv2LURQgwulU0yKCXAeYV9l0yakuCo7tushDXMhWAiOBMgUHoqNVxu_xVGuH6R02SzxSRzYoynpc0_0cStxZYYyxEnz1mZgenF4UOXhh5uHAj7jrecrK59ERnwzmBktlcnjBC68xpzUyY5UcvQfA&acatk=1ckph7ccr14l40rm&pub=4a1b367933fd867b19b072952f68dceb&vjs=3
extracting 4 job keywords...
['c++', 'python', 'sql', 'java'

In [7]:
# prevent the driver stopping due to the unexpectedAlertBehaviour.
webdriver.DesiredCapabilities.FIREFOX["unexpectedAlertBehaviour"] = "accept"
get_info = True
driver = webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
# set a page load time limit so that don't have to wait forever if the links are broken.
driver.set_page_load_timeout(15)

for k in range(1,num_pages + 1):
#this 5 pages reopen the browser is to prevent connection refused error.
    if k%5==0.0:
        driver.quit()
        driver=webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
        driver.set_page_load_timeout(15)

    current_url = start_url + "&start=" + str(k*10)
    print('extracting %d page of job searching results...' % k)
    resp = requests.get(current_url)
    current_soup = BeautifulSoup(resp.content, 'lxml')
    current_urls = current_soup.findAll('a',{'rel':'nofollow','target':'_blank'})
    current_urls = [link['href'] for link in current_urls]
    
    for i in range(len(current_urls)):
        get_info = True
        try:
            driver.get(base_url + current_urls[i])
        except TimeoutException:
            get_info = False
            continue 
        j = random.randint(1500,3200)/1000.0
        time.sleep(j) #waits for a random time
        if get_info:
            soup=BeautifulSoup(driver.page_source, 'lxml')
            print('extracting %d job keywords...' % i)
        single_job = keywords_f(soup)
        other_job = communication_skills_f(soup)
        print(single_job, other_job, len(soup))
        print(driver.current_url)
        job_keywords.append([driver.current_url,single_job, other_job])

            
driver.quit()

skills_dict = list()
for lss in job_keywords:
    for ls in lss:
        for word in ls:
            if word in soft_skills:
                skills_dict.append(('soft skills', word))
            elif word in program_languages:
                skills_dict.append(('program languages', word))
            elif word in analysis_software:
                skills_dict.append(('analysis software', word))
            elif word in bigdata_tool:
                skills_dict.append(('big data tooling', word))
            elif word in databases:
                skills_dict.append(('databases', word))
            elif word in machine_learning:
                skills_dict.append(('machine learning', word))
                
skills_count_dict = Counter(skills_dict)
skills_count_dict.keys()
df = pd.DataFrame()
df['Skill'] = skills_count_dict.keys()
df['Count'] = skills_count_dict.values()
df['Ranking'] = df['Count']/float(len(job_keywords))
df[['Category', 'Skill']] = df['Skill'].apply(pd.Series)
df= df[['Category', 'Skill', 'Count', 'Ranking']]
df

extracting 1 page of job searching results...
extracting 0 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=45d18c7995043d33&tk=1ckph8eir14l4437&from=serp&alid=3&advn=2327986768228300
extracting 1 job keywords...
['hadoop', 'r', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckph8hn014l469v&from=serp&alid=3&advn=2327986768228300
extracting 2 job keywords...
['spss', 'r', 'sql', 'python', 'excel'] 1
https://www.indeed.nl/vacature-bekijken?jk=2a3d7f6df1fca6ee&tk=1ckph8kc414l45fd&from=serp&alid=3&advn=9110344694364511
extracting 3 job keywords...
['matlab', 'c++', 'qlikview', 'spark', 'hadoop', 'spotfire', 'tableau', 'python', 'r', 'splunk', 'sap', 'sas', 'sql', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=40ebbf5c2c444ab3&from=serp&vjs=3
extracting 4 job keywords...
['spark', 'hadoop', 'python', 'r', 'java'] 1
https://www.indeed.nl/cmp/Xomnia/j

extracting 14 job keywords...
['hadoop', 'r', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckphd5m914l41ls&from=serp&alid=3&advn=2327986768228300
extracting 15 job keywords...
['java'] 1
https://www.indeed.nl/vacature-bekijken?jk=43cf6a283215a0f2&tk=1ckphd90m14l418q&from=serp&alid=3&advn=2421485629829485
extracting 4 page of job searching results...
extracting 0 job keywords...
['hadoop', 'r', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckphdcg214l46rs&from=serp&alid=3&advn=2327986768228300
extracting 1 job keywords...
['spss', 'r', 'sql', 'python', 'excel'] 1
https://www.indeed.nl/vacature-bekijken?jk=2a3d7f6df1fca6ee&tk=1ckphdfq014l466p&from=serp&alid=3&advn=9110344694364511
extracting 2 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=90f9baa202e07bbe&q=data+scientist&from=web&advn=4203524812083670&sjdu=e0gv2LURQgwulU0

extracting 13 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=90f9baa202e07bbe&from=serp&vjs=3
extracting 14 job keywords...
['java'] 1
https://www.indeed.nl/vacature-bekijken?jk=43cf6a283215a0f2&tk=1ckphhuib14bd0co&from=serp&alid=3&advn=2421485629829485
extracting 15 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=45d18c7995043d33&tk=1ckphi45n14bd3ba&from=serp&alid=3&advn=2327986768228300
extracting 7 page of job searching results...
extracting 0 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=45d18c7995043d33&tk=1ckphi8ic14bd22j&from=serp&alid=3&advn=2327986768228300
extracting 1 job keywords...
['hadoop', 'spss', 'r', 'sql', 'matlab', 'spark', 'python', 'scala', 'cassandra', 'bash'] 1
https://www.indeed.nl/vacature-bekijken?jk=de9bebdff1c8e0e0&tk=1ckphibv914bd4br&from=serp&alid=3&advn=7971013936929884
extracting 

extracting 13 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckphmjup14bd20g&from=serp&alid=3&advn=2327986768228300
extracting 14 job keywords...
['hadoop', 'spss', 'r', 'sql', 'matlab', 'spark', 'python', 'scala', 'cassandra', 'bash'] 1
https://www.indeed.nl/vacature-bekijken?jk=de9bebdff1c8e0e0&tk=1ckphmn8814bd32t&from=serp&alid=3&advn=7971013936929884
extracting 15 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=45d18c7995043d33&tk=1ckphmqo614bd31o&from=serp&alid=3&advn=2327986768228300
extracting 10 page of job searching results...
extracting 0 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=15081fe75b8b2efd&tk=1ckphn3r814k858g&from=serp&alid=3&advn=2327986768228300
extracting 1 job keywords...
['hadoop', 'spss', 'r', 'sql', 'matlab', 'spark', 'pyt

extracting 7 job keywords...
['python', 'r', 'sql', 'd3'] 1
https://www.indeed.nl/vacature-bekijken?jk=0a2f4132b983dec7&from=serp&vjs=3
extracting 8 job keywords...
[] 1
https://www.indeed.nl/vacature-bekijken?t=data+analist&jk=6a8c1098a5b97b57&vjs=3
extracting 9 job keywords...
['sql'] 1
https://www.indeed.nl/vacature-bekijken?jk=3ab5242a4a7f8ef5&from=serp&vjs=3
extracting 10 job keywords...
['tableau', 'r', 'sql'] 1
https://www.indeed.nl/vacature-bekijken?jk=68ae5b5ac3af4ec1&from=serp&vjs=3
extracting 11 job keywords...
['hadoop', 'spotfire', 'python', 'r', 'splunk', 'sql', 'd3'] 1
https://www.indeed.nl/vacature-bekijken?jk=af8db17dc637e0a6&from=serp&vjs=3
extracting 12 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=e28384c9608daedc&from=serp&vjs=3
extracting 13 job keywords...
[] 1
https://www.indeed.nl/cmp/Batavia-Biosciences-B.V./jobs/Scientist-Virus-Purification-c043221d0c08b4d0?vjs=3
extracting 14 job keywords...
['python', 'r'] 1
https://www.indeed

...

extracting 6 job keywords...
['cassandra', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=19009afd78c58bf1&from=serp&vjs=3
extracting 7 job keywords...
['scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=f31b4bee37a1d2b6&from=serp&vjs=3
extracting 8 job keywords...
['scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=a93acef9f1cce1d5&from=serp&vjs=3
extracting 9 job keywords...
['spark', 'hadoop', 'php', 'sql'] 1
https://www.indeed.nl/vacature-bekijken?jk=8432a4af6c435732&from=serp&vjs=3
extracting 10 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=90f9baa202e07bbe&q=data+scientist&from=web&advn=4203524812083670&sjdu=e0gv2LURQgwulU0yKCXAeYV9l0yakuCo7tushDXMhWAiOBMgUHoqNVxu_xVGuH6R02SzxSRzYoynpc0_0cStxZYYyxEnz1mZgenF4UOXhh5uHAj7jrecrK59ERnwzmBktlcnjBC68xpzUyY5UcvQfA&acatk=1ckpm75qr9ngqfe5&pub=4a1b367933fd867b19b072952f68dceb&vjs=3
extracting 11 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www

extracting 1 job keywords...
['r'] 1
https://www.indeed.nl/cmp/SeederDeBoer/jobs/Young-Data-Professional-04fe09d8330c6eb0?sjdu=L957Z3-jA_aTd9otCDNpUlW0qrkdxa7NajyPQf4Rd0DJ7CQZv_HeXAwHN_RJtzVimL_JYUSbVILCmRU_nk2bJssk7kG4Kqm5DFjPSxJc4pDauGQfjmVGZpLVJk1lzQdbwpRlpI3gegS82S_lgo6EXyqzVy5micdhFhYfnrTXjA6PY3WxnvetqLFse1SFKljCYK7sPVd-yxUISE3qtJI5kF0zRUgp-hgRYuuotzsERlq8l6QrtlO54nS2M_qQO52gXQGaLv2hjMZ-cxU0YuG_LURw84vJ41oiPGxnUhOCIteCvU_cl9pzcX7gf7lYNi33&vjs=3
extracting 2 job keywords...
['hadoop', 'spss', 'r', 'sql', 'matlab', 'spark', 'python', 'scala', 'cassandra', 'bash'] 1
https://www.indeed.nl/vacature-bekijken?jk=de9bebdff1c8e0e0&tk=1ckpmb63q9ngqbj1&from=serp&alid=3&advn=7971013936929884
extracting 3 job keywords...
['hadoop', 'javascript', 'spark', 'd3', 'python', 'scala'] 1
https://www.indeed.nl/vacature-bekijken?jk=45d18c7995043d33&tk=1ckpmb9l49ngqadg&from=serp&alid=3&advn=2327986768228300
extracting 4 job keywords...
['sql', 'sas', 'python'] 1
https://www.indeed.nl/vacature-bekijken?j

extracting 14 job keywords...
['java'] 1
https://www.indeed.nl/vacature-bekijken?jk=43cf6a283215a0f2&tk=1ckpmfn3k16vo32h&from=serp&alid=3&advn=2421485629829485
extracting 15 job keywords...
['r', 'matlab', 'python'] 1
https://www.indeed.nl/vacature-bekijken?jk=9261e612e6795cf5&tk=1ckpmfqpu16vo7tp&from=serp&alid=3&advn=2845914259062867
extracting 101 page of job searching results...
extracting 0 job keywords...
['python', 'r'] 1
https://www.indeed.nl/vacature-bekijken?jk=90f9baa202e07bbe&q=data+scientist&from=web&advn=4203524812083670&sjdu=e0gv2LURQgwulU0yKCXAeYV9l0yakuCo7tushDXMhWAiOBMgUHoqNVxu_xVGuH6R02SzxSRzYoynpc0_0cStxZYYyxEnz1mZgenF4UOXhh5uHAj7jrecrK59ERnwzmBktlcnjBC68xpzUyY5UcvQfA&acatk=1ckpmfv3616vo1de&pub=4a1b367933fd867b19b072952f68dceb&vjs=3
extracting 1 job keywords...
['r'] 1
https://www.indeed.nl/cmp/SeederDeBoer/jobs/Young-Data-Professional-04fe09d8330c6eb0?sjdu=L957Z3-jA_aTd9otCDNpUlW0qrkdxa7NajyPQf4Rd0DJ7CQZv_HeXAwHN_RJtzVimL_JYUSbVILCmRU_nk2bJssk7kG4Kqm5DFjPSxJc4pDauGQ

extracting 9 job keywords...
[] 1
https://www.indeed.nl/vacature-bekijken?jk=ac6e02643522cdd0&from=serp&vjs=3
extracting 10 job keywords...
['cassandra', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=19009afd78c58bf1&from=serp&vjs=3
extracting 11 job keywords...
['scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=f31b4bee37a1d2b6&from=serp&vjs=3
extracting 12 job keywords...
['scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=a93acef9f1cce1d5&from=serp&vjs=3
extracting 13 job keywords...
['spark', 'hadoop', 'php', 'sql'] 1
https://www.indeed.nl/vacature-bekijken?jk=8432a4af6c435732&from=serp&vjs=3
extracting 14 job keywords...
['java'] 1
https://www.indeed.nl/vacature-bekijken?jk=43cf6a283215a0f2&tk=1ckpmkmr716vo6b8&from=serp&alid=3&advn=2421485629829485
extracting 15 job keywords...
['r', 'matlab', 'python'] 1
https://www.indeed.nl/vacature-bekijken?jk=9261e612e6795cf5&tk=1ckpmkps016vo7oj&from=serp&alid=3&advn=2845914259062867
extracting 104 page of job se

extracting 4 job keywords...
['sql', 'sas', 'python'] 1
https://www.indeed.nl/vacature-bekijken?jk=5a9da55fc1315cb8&from=serp&vjs=3
extracting 5 job keywords...
[] 1
https://www.indeed.nl/vacature-bekijken?jk=b48f8c1d163affeb&from=serp&vjs=3
extracting 6 job keywords...
['spark', 'hadoop', 'python', 'scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=d54a5effbf6fcbaf&from=serp&vjs=3
extracting 7 job keywords...
['r'] 1
https://www.indeed.nl/vacature-bekijken?jk=76566f0f6406eb67&from=serp&vjs=3
extracting 8 job keywords...
['hadoop', 'javascript', 'spark', 'python', 'scala', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=c7d6a54aa42194c0&from=serp&vjs=3
extracting 9 job keywords...
[] 1
https://www.indeed.nl/vacature-bekijken?jk=ac6e02643522cdd0&from=serp&vjs=3
extracting 10 job keywords...
['cassandra', 'java'] 1
https://www.indeed.nl/vacature-bekijken?jk=19009afd78c58bf1&from=serp&vjs=3
extracting 11 job keywords...
['scala', 'java'] 1
https://www.indeed.nl/vacature-be

In [8]:
df.to_csv("skills_for_datascience.csv")